In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly as py
import plotly.graph_objs as go
import warnings
warnings.filterwarnings("ignore")


In [ ]:
%time
df=pd.read_csv("winequality-red.csv")

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()

In [ ]:
%time
df=spark.read.csv("winequality-red.csv",header=True,inferSchema=True)

In [ ]:
df.show(5)

In [ ]:
(df.describe().show())

In [ ]:
df.printSchema()

In [ ]:
df.count()

In [ ]:
len(df.columns)

In [ ]:
df.select(['quality']).distinct().show()

In [ ]:
from pyspark.sql.functions import col 
for columns in df.columns:
    print(columns,':',df.where(col(columns).isNull()).count())

In [ ]:
df=df.dropDuplicates()

In [ ]:
df.count()

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

In [ ]:
assembler=VectorAssembler(inputCols=df.columns, outputCol="feature")
assembler_df=assembler.transform(df)
assembler_df.show(5)

In [ ]:
# Calculate the correlation matrix
correlation_matrix = Correlation.corr(assembler_df, "feature").head()

# Extract the correlation matrix as a dense matrix
correlation_matrix = pd.DataFrame(correlation_matrix[0].toArray())

In [ ]:
correlation_matrix=correlation_matrix.rename(columns=dict(zip(correlation_matrix.columns,df.columns)))
correlation_matrix["values"]=correlation_matrix.columns
correlation_matrix.set_index('values',inplace=True)

In [ ]:
correlation_matrix

In [ ]:
plt.figure(figsize=(15,5))
sns.heatmap(correlation_matrix,annot=True)

In [ ]:
df_quality=df.groupBy("quality").count()
df_quality=df_quality.withColumnRenamed('count',"Total_count")

In [ ]:
df_quality_pd=df_quality.toPandas()

In [ ]:
sns.barplot(x=df_quality_pd.quality,y=df_quality_pd.Total_count)
plt.ylabel("Total Count of Quality")
plt.xlabel("Quality Score")
plt.show();

In [ ]:
df_pd=df.toPandas()
sns.pairplot(df_pd)